In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import math

In [95]:
df = pd.read_csv("dataset_output/lsa64_openpose/lsa64_dataset_cat_1.csv")

In [96]:
df = df.drop(["Unnamed: 0"], axis=1)

In [97]:
df

,category,video_name,frame,hand_0_0_x,hand_0_0_y,hand_0_0_z,hand_0_1_x,hand_0_1_y,hand_0_1_z,hand_0_2_x,...,pose_21_z,pose_22_x,pose_22_y,pose_22_z,pose_23_x,pose_23_y,pose_23_z,pose_24_x,pose_24_y,pose_24_z
0,1,001_001_001.mp4,0,0.559302,0.943834,0.361916,0.539392,0.984436,0.072649,0.527680,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,001_001_001.mp4,1,0.557995,0.944853,0.363025,0.538670,0.990662,0.072256,0.526957,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,001_001_001.mp4,2,0.557992,0.945950,0.366478,0.539255,0.989671,0.074984,0.527544,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,001_001_001.mp4,3,0.557876,0.945462,0.389607,0.538725,0.984666,0.079201,0.527699,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1,001_001_001.mp4,4,0.557848,0.945605,0.395565,0.538737,0.980610,0.080987,0.526575,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5895,1,001_010_005.mp4,113,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5896,1,001_010_005.mp4,114,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5897,1,001_010_005.mp4,115,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5898,1,001_010_005.mp4,116,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [75]:
landmarks_name = np.array([i for i in list(df.columns) if i not in ["Unnamed: 0", "Unnamed: 0.1", "Unnamed: 0.2", "category", "category_index", "video_name", "frame", "missing_hand", "missing_face"]])
landmarks_name = landmarks_name.reshape((int(landmarks_name.shape[0]/3), 3))

In [6]:
# GPT Code
def rotate_landmarks(landmarks, rotation_angle):
    # Assuming landmarks are in (x, y, z) format
    
    # Calculate the centroid (center) of the landmarks
#     centroid = np.mean(landmarks, axis=0)
    centroid = 0.5

    # Translate landmarks to the origin (center)
    translated_landmarks = landmarks - centroid
    # Create a rotation matrix for the given angle (in radians)
    rotation_matrix = np.array([[np.cos(rotation_angle), -np.sin(rotation_angle), 0],
                                 [np.sin(rotation_angle), np.cos(rotation_angle), 0],
                                 [0, 0, 1]])
    
    # Apply the rotation to each landmark
#     print(rotation_matrix)
    rotated_landmarks = np.dot(translated_landmarks, rotation_matrix)
    
    return rotated_landmarks + centroid

In [7]:
def zoom_landmarks(landmarks, zoom_factor):
    # Scale the landmarks by the zoom factor
    zoomed_landmarks = landmarks * zoom_factor
    
    return zoomed_landmarks

In [8]:
def translate_landmarks(landmarks, translation_vector):
    # Translate the landmarks by the given vector
    translated_landmarks = landmarks + translation_vector
    
    return translated_landmarks

In [111]:
def apply_transformation(frame, rotation, zoom, translate_x, translate_y):
    for lx, ly, lz in landmarks_name:
        result = rotate_landmarks(frame[[lx, ly, lz]], math.radians(rotation))
        frame[[lx, ly, lz]] = result
        result = zoom_landmarks(frame[[lx, ly, lz]], zoom)
        frame[[lx, ly, lz]] = result
        result = translate_landmarks(frame[[lx, ly, lz]], [translate_x, translate_x, 0])
        frame[[lx, ly, lz]] = result
    return frame

In [74]:
np.random.normal(0, 0.5, 10)

array([ 0.4013824 , -0.59702868, -0.54784934, -0.58209058, -0.11838624,
        0.44894116, -0.4087575 , -0.71535031, -0.44958185,  0.30856944])

In [123]:
sample_size = 0.7
samples_creation = 5
rotation_sigma = 5
zoom_sigma = 0.1
translate_x_sigma = 0.02
translate_y_sigma = 0.02

In [ ]:
# Old
df_augmented = pd.DataFrame(columns=df.columns)
for category in df["category"].unique():
    print("Processing category", category)
    df_category = df[df["category"] == category]
    videos = df_category["video_name"].unique()
    videos = np.random.choice(videos, int(len(videos) * sample_size))
    df_category = df_category[df_category["video_name"].isin(videos)]
    for i in range(samples_creation):
        df_sample = df_category.copy()
        df_sample["video_name"] = f"augmented{i}-" + df_sample["video_name"]
        cols_std = df_sample.drop(["frame", "video_name"], axis=1).agg(np.std)
        df_sample[col] = df_sample[col] * np.random.normal(0, 0.1)
#         for col in cols_std.index:
#             df_sample[col] = df_sample[col].apply(lambda i: np.random.normal(i, (cols_std[col] * 0.05)))
        df_augmented = pd.concat([df_augmented, df_sample])

In [124]:
df_augmented = pd.DataFrame(columns=df.columns)
for category in df["category"].unique():
    print("Processing category", category)
    df_category = df[df["category"] == category]
    videos = df_category["video_name"].unique()
    videos = np.random.choice(videos, int(len(videos) * sample_size))
    for i in range(samples_creation):
        for video in videos:
            df_video = df[df["video_name"] == video].copy()
            rotation = np.random.normal(0, rotation_sigma)
            zoom = np.random.normal(0, zoom_sigma) + 1
            translate_x = np.random.normal(0, translate_x_sigma)
            translate_y = np.random.normal(0, translate_y_sigma)
            df_video = apply_transformation(df_video, rotation, zoom, translate_x, translate_y)
#             for index, frame in df_video.iterrows():
#                 frame = apply_transformation(frame, rotation, zoom, translate_x, translate_y)
#                 df_video[index] = frame
            df_video["video_name"] = f"augmented{i}-" + df_video["video_name"]
            df_augmented = pd.concat([df_augmented, df_video])

Processing category 1


In [125]:
df_augmented

,category,video_name,frame,hand_0_0_x,hand_0_0_y,hand_0_0_z,hand_0_1_x,hand_0_1_y,hand_0_1_z,hand_0_2_x,...,pose_21_z,pose_22_x,pose_22_y,pose_22_z,pose_23_x,pose_23_y,pose_23_z,pose_24_x,pose_24_y,pose_24_z
5310,1,augmented0-001_010_001.mp4,0,-0.026263,-0.069610,0.000000,-0.026263,-0.069610,0.000000,-0.026263,...,0.0,-0.026263,-0.06961,0.0,-0.026263,-0.06961,0.0,-0.026263,-0.06961,0.0
5311,1,augmented0-001_010_001.mp4,1,-0.026263,-0.069610,0.000000,-0.026263,-0.069610,0.000000,-0.026263,...,0.0,-0.026263,-0.06961,0.0,-0.026263,-0.06961,0.0,-0.026263,-0.06961,0.0
5312,1,augmented0-001_010_001.mp4,2,-0.026263,-0.069610,0.000000,-0.026263,-0.069610,0.000000,-0.026263,...,0.0,-0.026263,-0.06961,0.0,-0.026263,-0.06961,0.0,-0.026263,-0.06961,0.0
5313,1,augmented0-001_010_001.mp4,3,-0.026263,-0.069610,0.000000,-0.026263,-0.069610,0.000000,-0.026263,...,0.0,-0.026263,-0.06961,0.0,-0.026263,-0.06961,0.0,-0.026263,-0.06961,0.0
5314,1,augmented0-001_010_001.mp4,4,-0.026263,-0.069610,0.000000,-0.026263,-0.069610,0.000000,-0.026263,...,0.0,-0.026263,-0.06961,0.0,-0.026263,-0.06961,0.0,-0.026263,-0.06961,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4333,1,augmented4-001_008_001.mp4,83,0.415343,0.847071,0.231921,0.394375,0.877689,0.051579,0.386247,...,0.0,0.024084,-0.04108,0.0,0.024084,-0.04108,0.0,0.024084,-0.04108,0.0
4334,1,augmented4-001_008_001.mp4,84,0.415056,0.848496,0.232541,0.391630,0.877693,0.047839,0.361236,...,0.0,0.024084,-0.04108,0.0,0.024084,-0.04108,0.0,0.024084,-0.04108,0.0
4335,1,augmented4-001_008_001.mp4,85,0.415051,0.848522,0.224687,0.391163,0.876708,0.046730,0.361246,...,0.0,0.024084,-0.04108,0.0,0.024084,-0.04108,0.0,0.024084,-0.04108,0.0
4336,1,augmented4-001_008_001.mp4,86,0.414457,0.849442,0.224891,0.391109,0.877671,0.047501,0.361876,...,0.0,0.024084,-0.04108,0.0,0.024084,-0.04108,0.0,0.024084,-0.04108,0.0


In [113]:
pd.concat([df, df_augmented]).to_csv("lsa64_dataset_best_80_fps_augmented.csv")

In [126]:
df_augmented.to_csv("datasets/lsa64_augmentation_test.csv")